In [2]:
from dotenv import load_dotenv

load_dotenv()

import json
import re
from pprint import PrettyPrinter

import polars as pl
from bs4 import BeautifulSoup
from httpx import Client
from pydantic import AnyHttpUrl
from tqdm.asyncio import tqdm

from douglas.internal.crawler import DouglasCrawler
from douglas.schemas import Product
from douglas.settings import settings

In [3]:
url = "https://www.douglas.de/de/c/gesicht/gesichtsmasken/feuchtigkeitsmasken/120308"
# url = "https://www.douglas.de/de/p/3001055831?variant=077163"  # Avocado
# url = "https://www.douglas.de/de/p/3000037624"  # with discount

## HTML scrape

In [3]:
client = Client()
res = client.get(
    url,
    headers={
        "Accept": "text/html",
        "User-Agent": settings.USER_AGENT,
    },
)

In [4]:
soup = BeautifulSoup(res.text, features="lxml")

In [10]:
[s.find("a").get("href") for s in soup.find_all("div", {"class": "product-tile"})]

['/de/p/5002082006',
 '/de/p/3000037624?variant=493205',
 '/de/p/3000037624?variant=493205',
 '/de/p/5010334127?variant=137660',
 '/de/p/5011530062',
 '/de/p/5003702027',
 '/de/p/5010535000',
 '/de/p/5010481013',
 '/de/p/3000063841',
 '/de/p/3001052450?variant=1104546',
 '/de/p/5010111057',
 '/de/p/5010704077',
 '/de/p/3001056394?variant=066496',
 '/de/p/3001001783',
 '/de/p/1069021166',
 '/de/p/3001001730?variant=803119',
 '/de/p/5002082007',
 '/de/p/3001052445',
 '/de/p/5011050004',
 '/de/p/5011656133',
 '/de/p/5011311015',
 '/de/p/3001055831?variant=077163',
 '/de/p/5002821015?variant=702648',
 '/de/p/5009004032',
 '/de/p/5010641022',
 '/de/p/5010301030',
 '/de/p/3001036090',
 '/de/p/5002082006',
 '/de/p/3001036090',
 '/de/p/3001001733?variant=803105',
 '/de/p/5011555034',
 '/de/p/3001052453',
 '/de/p/5010458013',
 '/de/p/3001056187',
 '/de/p/5011547062',
 '/de/p/5010555057',
 '/de/p/3001032505',
 '/de/p/5010574006',
 '/de/p/5010033015?variant=162578',
 '/de/p/5002787047',
 '/de/p/5

In [23]:
PrettyPrinter(indent=2).pprint(
    {
        "name": soup.find("span", attrs={"class": "header-name"}).text,
        "image": soup.find("img", attrs={"class": "image swiper-lazy"}).get(
            "data-lazy-src"
        ),
        "variants": [
            {
                "name": sou.find(
                    "div", attrs={"class": "product-detail__variant-name"}
                ).text,
                "price": re.search(
                    r"\d+\.\d+",
                    sou.find("span", attrs={"class": "product-price__price"})
                    .text.replace("\xa0", " ")
                    .replace(",", "."),
                    flags=re.I,
                ).group(),
            }
            for sou in soup.find_all(
                "div", attrs={"class": "product-detail__variant-row"}
            )
        ],
        "labels": [
            s.text
            for s in soup.find("div", attrs={"class": "product-labels"}).find_all(
                "span", {"class": "product-label__name"}
            )
        ],
        "properties": {
            sou.find_all("span")[0].text: sou.find_all("span")[1].text
            for sou in soup.find(
                "div", attrs={"data-testid": "product-detail-info__classifications"}
            ).find_all("div")
        },
        "description": soup.find(
            "div", attrs={"class": "truncate product-details__description"}
        ).text,
        "avg_rating": re.search(
            r"^\d+\.\d+",
            soup.find("span", attrs={"class": "ratings-info"}).text.replace(
                "\xa0", " "
            ),
            flags=re.I,
        ).group(),
        "total_ratings": re.search(
            r"\(\d+\)$",
            soup.find("span", attrs={"class": "ratings-info"}).text.replace(
                "\xa0", " "
            ),
            flags=re.I,
        )
        .group()
        .strip("()"),
    }
)

{ 'avg_rating': '4.5',
  'description': 'Verwöhnen Sie Ihre Haut mit der Avocado Nourishing Hydration '
                 'Mask und genießen Sie Ihren ganz persönlichen Masken-Moment! '
                 'Die unglaublich reichhaltige Gesichtsmaske von Kiehl’s '
                 'eignet sich dank der besonders milden und verträglichen '
                 'Formel für jeden Hauttyp, selbst für sensible Haut. '
                 'Angereichert mit Avocado-Frucht-Extrakt, Avocado-Öl und '
                 'Nachtkerzen-Öl, versorgt die einzigartige Maske die Haut '
                 'intensiv mit Feuchtigkeit und hinterlässt ein geschmeidig '
                 'zartes, straffes Hautgefühl. Für die cremige Textur ließen '
                 'sich die Beauty-Experten von Kiehl’s von frischen, reifen '
                 'Avocados inspirieren – so wird die Pflegeroutine zum '
                 'luxuriösen Wohlfühlerlebnis.',
  'image': 'https://media.douglas.de/medias/1p7Sek077163-0-dgl-DE.jpg?context=bWFz

## Full crawl

In [54]:
crawl = DouglasCrawler()
data = await crawl.product.search(url)
print(data.model_dump_json(indent=2))

{
  "page": 1,
  "page_size": 56,
  "total_pages": 12,
  "items": [
    "/de/p/5002082006",
    "/de/p/3000037624?variant=493205",
    "/de/p/3000037624?variant=493205",
    "/de/p/5010334127",
    "/de/p/5003702027",
    "/de/p/5010535000",
    "/de/p/5010481013",
    "/de/p/3000063841",
    "/de/p/5010111057",
    "/de/p/3001052450?variant=1104546",
    "/de/p/5010704077",
    "/de/p/3001056394?variant=066496",
    "/de/p/3001001783",
    "/de/p/5011656133",
    "/de/p/3001001730?variant=803119",
    "/de/p/5011050004",
    "/de/p/5002082007",
    "/de/p/3001052445",
    "/de/p/5011311015",
    "/de/p/5002821015?variant=702648",
    "/de/p/3001055831?variant=077163",
    "/de/p/5009004032",
    "/de/p/5010641022",
    "/de/p/5010301030",
    "/de/p/3001036090",
    "/de/p/5011555034",
    "/de/p/5002082006",
    "/de/p/3001052453",
    "/de/p/3001036090",
    "/de/p/5011618058",
    "/de/p/5010458013",
    "/de/p/3001056187",
    "/de/p/5011547062",
    "/de/p/5010555057",
    "/de/p

In [55]:
await crawl.product.get(
    str(
        AnyHttpUrl.build(
            scheme=settings.BASE_URL.scheme,
            host=settings.BASE_URL.host,
            path=data.items[0],
        )
    )
)

Product(ean='3614273010092', code='570368', url=Url('https://www.douglas.de//de/p/5002082006'), name='Aquasource Aqua Super Tuchmaske Glow', description='Soforthilfe für einen strahlenden GlowEntdecken Sie die Aqua Super Tuchmaske Glow – die perfekte Ergänzung zu Ihrem Aqua Glow Super Concentrate von Biotherm! Die intensiv feuchtigkeitsspendende Tuchmaske mit Vitamin C bringt fahle und dehydrierte Haut zum Strahlen, während die natürliche Hautbarriere gestärkt wird und die Haut so ebenmäßiger und leuchtender wirkt.\xa0Mehr zu diesem Produkt\xa0Tuchmaske mit Vitamin Chydratisiert & stärkt die natürliche Hautbarriereebenmäßige & strahlende Haut\xa0', average_rating=4.6, number_of_reviews=51, image=Url('https://media.douglas.de/medias/KQ7ALe570368-0-dgl-DE.png?context=bWFzdGVyfGltYWdlc3wzOTE4M3xpbWFnZS9wbmd8YUdGbUwyZ3lOQzgxTURNek1ERXpNVGt5TWprM05DOUxVVGRCVEdVMU56QXpOamhmTUY5a1oyd3RSRVV1Y0c1bnwxZTcxNGFmM2UzY2JlOTYxZjlkNWY5MGE0MDAwNDQyZDY2YWUzYzZiNDg3ZWQyMmEyZjIyYzI3OTFhNjBlMDk2&grid=true&i

In [ ]:
products: list[Product] = await tqdm.gather(
    *[
        crawl.product.get(
            str(
                AnyHttpUrl.build(
                    scheme=settings.BASE_URL.scheme,
                    host=settings.BASE_URL.host,
                    path=p,
                )
            )
        )
        for p in data.items
    ]
)

print(
    json.dumps(
        [p.model_dump(mode="json") for p in products], indent=2, ensure_ascii=False
    )
)

AttributeError: module 'polars' has no attribute 'from_pydantic'

In [ ]:
df = pl.from_dicts([p.model_dump(mode="json") for p in products]).unique(
    ["ean"], keep="first"
)
df

In [ ]:
variants_df = (
    df.select(product_id=pl.col("ean"), variants=pl.col("variants"))
    .explode("variants")
    .unnest("variants")
)
variants_df

In [51]:
classifications_df = (
    df.select(
        product_id=pl.col("ean"),
        classifications=pl.col("classifications"),
    )
    .explode("classifications")
    .unnest("classifications")
)
classifications_df

product_id,key,value
str,str,str
"""3666057192012""","""Art-Nr.""","""1122645"""
"""3666057192012""","""Konsistenz""","""Creme"""
"""3666057192012""","""Hauttyp""","""Alle Hauttypen"""
"""3666057192012""","""Finish""","""natürlich"""
"""3666057192012""","""Eigenschaft""","""pflegend, feuchtigkeitsspenden…"
…,…,…
"""5057566631778""","""Art-Nr.""","""1079403"""
"""5057566631778""","""Hauttyp""","""Alle Hauttypen"""
"""5057566631778""","""Eigenschaft""","""feuchtigkeitsspendend, straffe…"
